<a href="https://colab.research.google.com/github/rromerov/Proyecto_Integrador/blob/main/Avance1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instituto Tecnológico y de Estudios Superiores de Monterrey
## Maestría en Inteligencia Artificial Aplicada
### Proyecto Integrador (Gpo 10) - TC5035.10

### **Proyecto: Diseño Acelerado de Fármacos**

### Avance 1: Análisis exploratorio de datos

#### **Docentes:**
- Dra. Grettel Barceló Alonso - Profesor Titular
- Dr. Luis Eduardo Falcón Morales - Profesor Titular
- Dr. Ricardo Ambrocio Ramírez Mendoza  – Profesor Tutor

#### **Miembros del equipo:**
- Ernesto Enríquez Rubio - A01228409
- Roberto Romero Vielma - A00822314
- Herbert Joadan Romero Villarreal –  A01794199

# Análisis exploratorio de los datos

## Instalar librerias faltantes

In [ ]:
! pip install chembl_webresource_client
! pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
! pip install rdkit

## Importar librerias

In [ ]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
from google.colab import drive
import locale
import condacolab
import sys
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.random import seed
from numpy.random import randn
from scipy.stats import mannwhitneyu

**Compuestos inhibidores de la proteina VEFG**

* Pazopanib: Es un inhibidor multiquinasa con biodisponibilidad oral. Actúa sobre los tres tipos de receptores del factor de crecimiento vascular endotelial (VEGFR-1, VEGFR-2 y VEGFR-3),
* Sunitinib: Actúa bloqueando los receptores del factor de crecimiento derivado de plaquetas (PDGFRα y PDGFRβ), los receptores VEGF (VEGFR1, VEGFR2 y VEGFR3), el receptor de factor de células madre (KIT), la tirosin-quinasa 3 tipo Fms (FLT3), el factor estimulador de colonias (CSF-1R) y el receptor del factor de crecimiento epidérmico (EGFR).
* Bevacizumab: Al unirse al VEGF, bevacizumab inhibe su unión a los receptores Flt-1 (VEGFR-1) y KDR (VEGFR-2) presentes en la superficie de las células endoteliales.
* Sorafenib:  Inhibe varios receptores de factores de crecimiento, como las kinasas de tirosina. Algunos de estos receptores incluyen:
PDGFR-beta,VEGFR-2 (receptor 2 del factor de crecimiento endotelial vascular), VEGFR-3
* Regorafenib:  Afecta a múltiples quinasas, incluyendo aquellas involucradas en la angiogénesis, mientras que el VEGF 165 desempeña un papel crítico en la promoción de la formación de vasos sanguíneos.
* Cabozantinib: cabozantinib actúa como un inhibidor de VEGF y MET, lo que lo convierte en una opción terapéutica para ciertos tipos de cáncer.
* Lenvatinib: Se dirige selectivamente a la actividad quinasa de varios receptores, incluyendo los del factor de crecimiento del endotelio vascular (VEGF): VEGFR1 (FLT1), VEGFR2 (KDR) y VEGFR3 (FLT4).

# ChEMBL IDs



In [ ]:
# ChEMBL IDs
drugs_for_vegf_protein_ids = {
    'pazopanib': 'CHEMBL477772',
    'sunitinib': 'CHEMBL535',
    # 'bevacizumab': 'CHEMBL1201583',
    'sorafenib': 'CHEMBL1336',
    'regorafenib': 'CHEMBL1946170',
    'cabozantinib': 'CHEMBL2105717',
    'lenvatinib': 'CHEMBL1289601',
    'vandetanib': 'CHEMBL24828',
    'axitinib' :'CHEMBL1289926',
    'foretinib': 'CHEMBL1230609',
    'tivozanib': 'CHEMBL1289494',
    'dovitinib': 'CHEMBL522892',
    'orantinib': 'CHEMBL274654',
    'nintedanib': 'CHEMBL502835',
    'vatalanib': 'CHEMBL101253',
    'telatinib': 'CHEMBL2079588',
    'motesanib': 'CHEMBL572881',
    'brivanib': 'CHEMBL377300',
    'linifanib': 'CHEMBL223360',
    'regorafenib': 'CHEMBL1946170'}

In [ ]:

# Obtener información molecular
molecule = new_client.molecule
drugs_can_smiles = {}
for drug_name, drug_chembl in drugs_for_vegf_protein_ids.items():
    drugs_can_smiles.update({drug_name: molecule.get(drug_chembl).get('molecule_structures').get('canonical_smiles')})
    # Print out the results
    print(drug_name, ' > ', drugs_can_smiles[drug_name])

# Inhibidores del receptor del factor de crecimiento endotelial vascular

In [ ]:
# Accede al atributo 'mechanism'
mechanism_client = new_client.mechanism

# Busca el ID del mecanismo "Vascular endothelial growth factor receptor inhibitor"
# Si no conoces el ID de antemano, puedes buscarlo filtrando por mecanismo de acción
mechanism_name = 'Vascular endothelial growth factor receptor inhibitor'
mechanism_info = mechanism_client.filter(mechanism_of_action=mechanism_name)


In [ ]:
# Obtener los compuestos ChMBL que son inhibidores
inhi_chmbl_ids = [mechanism_info[i]['molecule_chembl_id'] for i in range(len(mechanism_info))]

In [ ]:
# Obtener información molecular
molecule = new_client.molecule
drugs_can_smiles = {}
for drug_chembl in inhi_chmbl_ids:
    try:
      molecule_chembl = molecule.get(drug_chembl)
      molecule_structure = molecule_chembl.get('molecule_structures')
      molecule_canon_smiles = molecule_structure.get('canonical_smiles')
      molecule_name = molecule_structure.get('molfile').split('\n')[-1].lower()
      drugs_can_smiles.update({molecule_name: molecule_canon_smiles})
      # Print out the results
      print(molecule_name, ' > ', drugs_can_smiles[molecule_name])
    except:
      print(f'Compuesto {drug_chembl} no fue encontrado.')

## Buscar proteina target, en este caso VEGF165

In [ ]:
# Búsqueda de VEGF165 target
%load_ext cudf.pandas
target = new_client.target
target_query = target.search('Neuropilin-1')
targets = pd.DataFrame.from_dict(target_query)
targets

## Selecionar y recuperar los datos de bioactividad para la proteina VEGF165

In [ ]:
selected_target = 'CHEMBL5174'

In [ ]:
# Filtrar el dataset para solo tener la fila donde el target_chembl_id sea igual a CHEMBL5174
selected_target_row = targets[targets['target_chembl_id'] == selected_target]
selected_target_row

Aqui vamos a recuperar los datos de bioactividad para la proteina VEGF165

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type='IC50')

In [ ]:
pd.set_option('display.max_columns', None)
# Guardar response como dataframe
df = pd.DataFrame.from_dict(res)
df.head()

In [ ]:
records_df = len(df)
print(f'Number of records in the dataset: {records_df}')

## Descripción de columnas relevantes dentro del dataframe:

* Standard Value: Es la potencia de la droga, mientras menor sea el valor más eficaz es, debido a que un mayor significa que se requiere de una mayor cantidad de droga para tener el mismo efecto

In [ ]:
# Verificar que solo se cuente con registros de IC50
df.standard_type.unique()

In [ ]:
# Visualizar rango de valores de standard value
df.standard_value.unique()

# Guardar resultados en Google Drive

In [ ]:
# Cargar Google Drive al notebook
drive.mount('/content/drive')

In [ ]:
locale.getpreferredencoding = lambda: "UTF-8"
# Crear carpeta dentro de google drive llamada data
! mkdir '/content/drive/My Drive/Colab Notebooks/data'

In [ ]:
# Subir el csv a la carpeta destino
df.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data.csv', index=False)

In [ ]:
# Verificar que el archivo se encuentra en la carpeta destino
! ls -l '/content/drive/My Drive/Colab Notebooks/data'

Ver contenido de **bioactivity_data.csv**

In [ ]:
! head '/content/drive/My Drive/Colab Notebooks/data/bioactivity_data.csv'

## Manejo de valores faltantes
Dado que nos interesa conocer el standard value, los registros que no cuenten con esta información serán eliminados

In [ ]:
df2 = df[df.standard_value.notna()]
df2

In [ ]:
records_df2 = len(df2)
preserved_info = round((records_df2/records_df)*100,2)
print(f'Number of records in df2 {records_df2}, percentage of information preserved: {preserved_info}')

In [ ]:
# Verificar la cantidad de registros de canonical smiles
canon_smiles_count = sum(df2.canonical_smiles.notnull())
canon_smiles_count

In [ ]:
# Verificar que la columna canonical_smiles cuente con valores unicos
unique_canon_smiles_count = len(df2.canonical_smiles.unique())
percentage_unique_canon_smiles = round(unique_canon_smiles_count/canon_smiles_count*100,2)
print(f'Number of unique canonical_smiles records in df2: {unique_canon_smiles_count}, percentage of unique records: {percentage_unique_canon_smiles}')

### Eliminar canonical_smiles duplicadas

In [ ]:
df2_unique_can_smiles = df2.drop_duplicates(['canonical_smiles'])
df2_unique_can_smiles

## Preprocesamiento de datos de bioactividad

### Combinar 3 columnas (molecule_chembl_id,canonical_smiles,standard_value) y bioactivity_class en un DataFrame

In [ ]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_unique_can_smiles[selection]
df3

Guardar resultados en un archivo CSV

In [ ]:
# Subir el csv a la carpeta destino
df3.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_preprocessed.csv', index=False)

In [ ]:
# Verificar cambios
! ls '/content/drive/My Drive/Colab Notebooks/data/'

### Etiquetado de componentes
Los datos de bioactividad estan en la unidad IC50. Compuestos con menos de 1000 nM será considerados como **activos** mientras aquellos que sean mayores a 10,000 nM serán considerados como **inactivos**. Todos los valores que caen dentro de 1,000 y 10,000 nM serán clasificados como **intermedios**.

In [ ]:
# Leer csv con datos preprocesados
df4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_preprocessed.csv')

In [ ]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [ ]:
# Concatenar lista generada como una serie de pandas y agregarla al df
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

Guardad dataframe como archivo csv

In [ ]:
# Subir el csv a la carpeta destino
df5.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_curated.csv', index=False)

## Cargar datos de bioactividad

In [ ]:
# Leer el archivo con los datos curados
df6 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_curated.csv')
df6

## Calcular descriptores Lipinski

### Regla de Lipinski


La regla de Lipinski establece los siguientes criterios para evaluar la idoneidad de una molécula como candidato a fármaco:

1. **Peso molecular (MW):** MW < 500
2. **LogP (coeficiente de partición octanol-agua):** LogP < 5
3. **Número de donantes de hidrógeno (HBD):** HBD < 5
4. **Número de aceptores de hidrógeno (HBA):** HBA < 10



In [ ]:
def lipinski(smiles, verbose=False):

  moldata = []
  for element in smiles:
    mol = Chem.MolFromSmiles(element)
    moldata.append(mol)

  baseData = np.arange(1,1)
  i = 0
  for mol in moldata:

    desc_MolWt = Descriptors.MolWt(mol)
    desc_MolLogP = Descriptors.MolLogP(mol)
    desc_NumHDonors = Lipinski.NumHDonors(mol)
    desc_NumAcceptors = Lipinski.NumHAcceptors(mol)

    row = np.array([desc_MolWt,
                    desc_MolLogP,
                    desc_NumHDonors,
                    desc_NumAcceptors])

    if i==0:
      baseData = row
    else:
      baseData = np.vstack([baseData, row])
    i = i+1

  columNames = ['MW','LogP','NumHDonors','NumHAcceptors']
  descriptors = pd.DataFrame(data=baseData, columns = columNames)

  return descriptors

In [ ]:
df_lipinski = lipinski(df6.canonical_smiles)

## Combinar DataFrames

In [ ]:
# Visualizar el dataframe con los descriptores calculados
df_lipinski

In [ ]:
combined_df = pd.concat([df6, df_lipinski], axis=1)
combined_df

## Convertir IC50 a pIC50

Para contar con datos más uniformes, se convirtió **IC50** a su escala logaritmica negativa, lo cual esencialemnte es ${-\log_{10}(IC_{50})}$

Se definió una función **pIC50** la cual aceptará un dataframe como entrada y hará lo siguiente:

* Tomar los valores de IC50 de la columna **standard_value** y los convertirá de nM a M mediante la multiplicación del valor por ${10^{-9}}$.
* Tomar el valor molar y aplicar ${-\log_{10}}$
* Borrar la columna de **standard_value** y crear una nueva columna llamada **pIC50**.

In [ ]:
def pIC50(input):
  pIC50 = []
  for i in input['standard_value_norm']:
    molar = i*(10**-9) # Convierte nM a M
    pIC50.append(-np.log10(molar))

  input['pIC50'] = pIC50
  x = input.drop(columns='standard_value_norm')

  return x

Los valores mayores a 100,000,000 se quedarán en 100,000,000, de no hacerlo así los valores logaritmicos negativos se convertirán en negativos.

In [ ]:
combined_df.standard_value.describe()

En este caso no es necesario pero se debe implementar esa lógica para evitar cualquier tipo de problema en el futuro.

In [ ]:
def norm_value(input):
    norm = []

    for i in input['standard_value']:
        if i > 100000000:
          i = 100000000
        norm.append(i)

    input['standard_value_norm'] = norm
    x = input.drop(columns='standard_value')

    return x

Primero aplicaremos la función **norm_value** para que los valores dentro de la columna **standard_value** sean normalizados.

In [ ]:
df_norm = norm_value(combined_df)

El siguiente paso es convertir la columna IC50 a pIC50

In [ ]:
df_final = pIC50(df_norm)
df_final

In [ ]:
df_final.pIC50.describe()

## Eliminar la clase bioactiva intermedia

In [ ]:
df_2classes = df_final[df_final['class'] != 'intermediate']
df_2classes

In [ ]:
# Guardar dataframe en archivo csv
df_final.to_csv('/content/drive/My Drive/Colab Notebooks/data/bioactivity_data_2class_pIC50.csv', index=False)

## Análisis exploratorio de los datos (Análisis del Espacio Químico) mediante descriptores Lipinski

### Distribución de las dos clases de bioactividad

In [ ]:
sns.set(style='ticks')
plt.figure(figsize=(5.5, 5.5))

sns.countplot(x='class', data=df_2classes, edgecolor='black', hue='class')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('Frequency', fontsize=14, fontweight='bold')

plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_bioactivity_class.pdf')

### Gráfica de dispersión

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.scatterplot(x='MW', y='LogP', data=df_2classes, hue='class', size='pIC50', edgecolor='black', alpha=0.7)

plt.xlabel('MW', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_MW_vs_LogP.pdf')

### Diagrama de caja
#### Valor de pIC50

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'pIC50', data = df_2classes, hue = 'class')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('pIC50 value', fontsize=14, fontweight='bold')

plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_ic50.pdf')

##### Análisis estadístico
**Prueba U Mann-Whitney**


In [ ]:
def mannwhitney(descriptor, verbose=False):

  # Generador de semilla aleatoria
    seed(1)

  # Activos e inactivos
    selection = [descriptor, 'class']
    df = df_2classes[selection]
    active = df[df['class'] == 'active']
    active = active[descriptor]

    selection = [descriptor, 'class']
    df = df_2classes[selection]
    inactive = df[df['class'] == 'inactive']
    inactive = inactive[descriptor]

  # Comparar muestras
    stat, p = mannwhitneyu(active, inactive)

  # Interpretar
    alpha = 0.05
    if p > alpha:
      interpretation = 'Same distribution (fail to reject H0)'
    else:
      interpretation = 'Different distribution (reject H0)'

    results = pd.DataFrame({'Descriptor':descriptor,
                            'Statistics':stat,
                            'p':p,
                            'alpha':alpha,
                            'Interpretation':interpretation}, index=[0])
    filename = 'mannwhitneyu_' + descriptor + '.csv'
    results.to_csv(f'/content/drive/My Drive/Colab Notebooks/data/{filename}')

    return results

In [ ]:
mannwhitney('pIC50')

#### MW

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'MW', data = df_2classes, hue = 'class')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('MW', fontsize=14, fontweight='bold')

plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_MW.pdf')

In [ ]:
mannwhitney('MW')

#### LogP

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'LogP', data = df_2classes, hue = 'class')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('LogP', fontsize=14, fontweight='bold')

plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_LogP.pdf')

In [ ]:
mannwhitney('LogP')

#### NumHDonors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'NumHDonors', data = df_2classes, hue = 'class')

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHDonors', fontsize=14, fontweight='bold')

plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_NumHDonors.pdf')

In [ ]:
mannwhitney('NumHDonors')

#### NumHAcceptors

In [ ]:
plt.figure(figsize=(5.5, 5.5))

sns.boxplot(x = 'class', y = 'NumHAcceptors', data = df_2classes)

plt.xlabel('Bioactivity class', fontsize=14, fontweight='bold')
plt.ylabel('NumHAcceptors', fontsize=14, fontweight='bold')

plt.savefig('/content/drive/My Drive/Colab Notebooks/data/plot_NumHAcceptors.pdf')

In [ ]:
mannwhitney('NumHAcceptors')

#### Guardar en un archivo comprimido los archivos generados

In [ ]:
# Guardar archivos en un archivo zip
!zip -r /content/drive/My\ Drive/Colab\ Notebooks/data/results.zip /content/drive/My\ Drive/Colab\ Notebooks/data/*.pdf /content/drive/My\ Drive/Colab\ Notebooks/data/*.csv

In [ ]:
# Verificar cambios
! ls '/content/drive/My Drive/Colab Notebooks/data/'